<h1>Connect to MSU ODM2 database get time series results and run some statistics. Then copy dataloggerfiles for Manistee sensors. </h1>
<p>1) Import odm2api to enable a database connection</p>

In [1]:
import numpy
from odm2api.ODMconnection import dbconnection
import odm2api.ODM2.services.readService as odm2
from odm2api.ODM2.models import *

# ! conda create -n odm2apienv -c odm2 python=2.7 odm2api
# ! conda info odm2api
# ! source activate odm2apienv

# ! conda list

# run these ones
! source activate python2
! pip install --upgrade git+git://github.com/miguelcleon/ODM2PythonAPI@master

<p>2) connect to the database and create a read service for it.</p>

In [2]:
session_factory = dbconnection.createConnection('postgresql', 'x.x.x.x', 'msu', 
                                                'user', 'pwd')
read = odm2.ReadODM2(session_factory)
DBSession = session_factory.getSession()

<p>3) get time series result values for a result.<p>

In [3]:
tempResultValues = read.getResultValues(resultids=[125], starttime='2015-9-1', 
                                                 endtime= '2015-9-10')

<p>4) run some basic statistics using the numpy library. M-6 Manistee River temperature.</p>

In [4]:
print("M-6 Manistee River")
print("maximum of values")
print(numpy.amax(tempResultValues))
print("minimum of values")
print(numpy.amin(tempResultValues)) 
print("mean value")
varmean = numpy.mean(tempResultValues['datavalue'])
print(numpy.mean(tempResultValues['datavalue']))
print("standard deviation of values")
varstd = numpy.std(tempResultValues['datavalue'])
print(numpy.std(tempResultValues['datavalue'])) 

M-6 Manistee River
maximum of values
valueid                                       2675059
resultid                                          125
datavalue                                        19.7
valuedatetime                     2015-09-10 00:00:00
valuedatetimeutcoffset                              4
censorcodecv                             Not censored
qualitycodecv                                    Good
timeaggregationinterval                            15
timeaggregationintervalunitsid                      8
dtype: object
minimum of values
valueid                                       2673331
resultid                                          125
datavalue                                        14.5
valuedatetime                     2015-09-01 00:00:00
valuedatetimeutcoffset                              4
censorcodecv                             Not censored
qualitycodecv                                    Good
timeaggregationinterval                            15
timeaggregati

In [5]:
template_dataloggerfile =  DBSession.query(DataLoggerFiles).filter_by(DataLoggerFileID=9).one()
print(template_dataloggerfile)
dlfcs = DBSession.query(DataLoggerFileColumns).filter_by(DataLoggerFileID=9)
#for dlfc in dlfcs:
#    print(dlfc)

<DataLoggerFiles({'DataLoggerOutputFileLink': u'dataloggerfiles/38099_001.PRN', 'ProgramID': 1, 'DataLoggerOutputFileDescription': u'M1', 'DataLoggerFileName': u'M1', 'DataLoggerFileID': 9})>


No sampling feature with id 4, sampling feature with id 3 'A-2 Bradford Creek' already has a data logger file.  
A-2 bradford creek is here: 
http://msu-odm2admin.cuahsi.org/MSU/odm2admin/samplingfeatures/3/change/ 

In [6]:

for i in range(2,20):
    print(i)
    if i == 3 or i==4:
        continue
    sampling_feature =  DBSession.query(SamplingFeatures).filter_by(SamplingFeatureID=i).one()
    print(sampling_feature.SamplingFeatureDescription)

2
A-1 Kolke Creek Kolke Creek Circle Trail
3
4
5
A-4 AuSable River (Main, W. of Grayling) MI-72 West
6
A-6 E. Branch AuSable River (Lower) Wilcox Bridge Road
7
A-7 E. Branch AuSable River (Mid) W. Karen Lake Road
8
A-8 Tributary to E. Branch AuSable River Lewiston Grade
9
A-9 E. Branch AuSable River (Upper) CR 612
10
A-11 AuSable River (Main) S. Wakeley Bridge Road
11
A-13 S. Branch AuSable River (Upper) Chase Bridge Road
12
A-14 AuSable River (Main) McMasters Bridge Road
13
A-15 N. Branch AuSable River  Flashlight Drive
14
A-16 N. Branch AuSable River (Mid) CR 612 (Lovells)
15
A-18 West Branch Big Creek M-72 Durfee Road
16
A-19 Middle Branch Big Creek Walsh Road
17
A-20 Ausable River  Red Oak Road
18
A-21 Big Creek Randall Road
19
A-23 Rabbit and Beaver Creeks Stephan Bridge Road


In [7]:
new_data_logger_file_ids = []

for i in range(2,20): #start range at when running for real 2
    if i == 3 or i==4:
        continue
    print(i)
    sampling_feature =  DBSession.query(SamplingFeatures).filter_by(SamplingFeatureID=i).one()
    template_dataloggerfile =  DBSession.query(DataLoggerFiles).filter_by(DataLoggerFileID=9).one()
    #template_dataloggerfile.save()
    new_dataloggerfile =DataLoggerFiles(ProgramID=1, DataLoggerFileName=sampling_feature.SamplingFeatureCode, 
                                        DataLoggerOutputFileDescription=sampling_feature.SamplingFeatureName,
                                       DataLoggerOutputFileLink ='')
    DBSession.add(new_dataloggerfile)
    DBSession.commit()
    new_data_logger_file_ids.append(new_dataloggerfile.DataLoggerFileID)
    results = DBSession.query(Results).join(FeatureActions).filter(FeatureActions.SamplingFeatureID==i)
    #for result in results:
    #    print(result)
    #results = Results.objects.filter(featureactionid__samplingfeatureid=i)
    
    dlfcs = DBSession.query(DataLoggerFileColumns).filter_by(DataLoggerFileID=9)
    for dlfc in dlfcs:
        print(dlfc)
        for result in results:
            old_dlfc_result = DBSession.query(Results).filter_by(ResultID=dlfc.ResultID).one()
            if result.VariableID == old_dlfc_result.VariableID:
                new_dlfc = DataLoggerFileColumns(ResultID= result.ResultID,
                                                 DataLoggerFileID = new_dataloggerfile.DataLoggerFileID, 
                                                 InstrumentOutputVariableID = dlfc.InstrumentOutputVariableID,
                                                ColumnLabel = dlfc.ColumnLabel,
                                                ColumnDescription = dlfc.ColumnDescription,
                                                MeasurementEquation = dlfc.MeasurementEquation,
                                                ScanInterval = dlfc.ScanInterval,
                                                ScanIntervalUnitsID = dlfc.ScanIntervalUnitsID,
                                                RecordingInterval = dlfc.RecordingInterval,
                                                RecordingIntervalUnitsID = dlfc.RecordingIntervalUnitsID,
                                                AggregationStatisticCV = dlfc.AggregationStatisticCV)
                DBSession.add(new_dlfc)
                DBSession.commit()
        

2
<DataLoggerFileColumns({'ScanIntervalUnitsID': None, 'DataLoggerFileColumnID': 44, 'RecordingIntervalUnitsID': 8, 'InstrumentOutputVariableID': 11, 'ScanInterval': None, 'MeasurementEquation': u'', 'ColumnDescription': u'datetime', 'RecordingInterval': 15.0, 'ColumnLabel': u'Date and Time', 'ResultID': 99L, 'DataLoggerFileID': 9, 'AggregationStatisticCV': None})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
5
<DataLoggerFileColumns({'ScanIntervalUnitsID': None, 'DataLoggerFileColumnID': 44, 'RecordingIntervalUnitsID': 8, 'InstrumentOutputVariableID': 11, 'ScanInterval': None, 'MeasurementEquation': u'', 'ColumnDescription': u'datetime', 'RecordingInterval': 15.0, 'ColumnLabel': u'Date and Time', 'ResultID': 99L, 'DataLoggerFileID': 9, 'AggregationStatisticCV': None})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
<DataLoggerFileColumns({})>
<DataLoggerF

Only run the next section if you want to delete the dataloggerfiles we just created. 

In [8]:
new_data_logger_files = DBSession.query(DataLoggerFiles).filter(
        DataLoggerFiles.DataLoggerFileID.in_(new_data_logger_file_ids))
for new_data_logger_file in new_data_logger_files:
    DBSession.delete(new_data_logger_file)
DBSession.commit()